In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

In [2]:
#leeftijdsgroepen = SimpleNamespace()
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2021-10-08@13-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2021-10-08@13-15.csv


loading rivm/COVID-19_casus_landelijk-2021-10-08@13-15.csv.gz


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2021-10-08 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2021-10-08 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2021-10-08 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2021-10-08 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,No,NaN,GGD Hollands-Noorden
4,2021-10-08 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


In [3]:
@run
def cell():
  global bevolking
  bevolking = CBS.bevolking(leeftijdsgroepen=True)

In [4]:
# Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.
# vervang <50 en Unknown door Onbekend
@run
def cell():
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  cohorten = list(bevolking.index) + ['Onbekend']
  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  factor = bevolking.to_dict()['per 100k']
  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  # rood -> groen
  cdict = {
    'red':   ((0.0, 0.0, 0.0),   # no red at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
    'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0)),  # no green at 1
    'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0))   # no blue at 1
  }
  cm = colors.LinearSegmentedColormap('GnRd', cdict)
  # geel -> paars
  cm = sns.color_palette('viridis_r', as_cmap=True)
  display(tabel
    .fillna(0)
    .head()
    .round(1)
    .reset_index(drop=True)
    .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
  )

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2021-10-08 00:00:00,02/10-08/10,35.100000,9087.000000,Positief getest,978.000000,1678.000000,1512.000000,1450.000000,1259.000000,962.000000,639.000000,356.000000,192.000000,61.000000,0.000000,582,1000,901,864,750,573,380,212,114,36,0
1,p001,1,2021-10-08 00:00:00,25/09-01/10,35.600000,11295.000000,Positief getest,1111.000000,2001.000000,1888.000000,1895.000000,1565.000000,1221.000000,862.000000,461.000000,241.000000,50.000000,0.000000,555,1000,943,947,782,610,430,230,120,24,0
2,p002,2,2021-10-08 00:00:00,18/09-24/09,33.700000,11620.000000,Positief getest,1396.000000,2361.000000,1820.000000,1868.000000,1590.000000,1145.000000,751.000000,421.000000,212.000000,55.000000,1.000000,591,1000,770,791,673,484,318,178,89,23,0
3,p003,3,2021-10-08 00:00:00,11/09-17/09,32.200000,13376.000000,Positief getest,1909.000000,2742.000000,2149.000000,2105.000000,1758.000000,1260.000000,765.000000,428.000000,203.000000,56.000000,1.000000,696,1000,783,767,641,459,278,156,74,20,0
4,p004,4,2021-10-08 00:00:00,04/09-10/09,31.700000,16091.000000,Positief getest,2354.000000,3265.000000,2773.000000,2593.000000,2018.000000,1420.000000,890.000000,463.000000,263.000000,51.000000,1.000000,720,1000,849,794,618,434,272,141,80,15,0


In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


Key                     object
Weken terug              int64
Datum                   object
Periode                 object
Gemiddelde leeftijd    float64
Totaal                 float64
Type                    object
0-9                    float64
10-19                  float64
20-29                  float64
30-39                  float64
40-49                  float64
50-59                  float64
60-69                  float64
70-79                  float64
80-89                  float64
90+                    float64
Onbekend               float64
c0-9                     int64
c10-19                   int64
c20-29                   int64
c30-39                   int64
c40-49                   int64
c50-59                   int64
c60-69                   int64
c70-79                   int64
c80-89                   int64
c90+                     int64
cOnbekend                int64
dtype: object
updating knack


...

updating: [{'Key': 1, 'Updating Leeftijdsgroep': 'false'}]
Key                         int64
Updating Leeftijdsgroep    object
dtype: object


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.10it/s]

100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 0, update: 1, delete: 0, backoff: 0
